<a href="https://colab.research.google.com/github/zahraDehghanian97/stock_market/blob/master/Arima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
! curl -L https://api.github.com/repos/zahraDehghanian97/stock_market/tarball --output repo.tar
! tar xf repo.tar --wildcards "*/tickers_data/*.csv" --strip-components=2 
! rm -rf repo.tar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  123k    0  123k    0     0   498k      0 --:--:-- --:--:-- --:--:--  498k


In [25]:


def load_df_from_csv(dir_name:str) -> dict:
  stocks = {}
  for file_name in sorted(glob.glob(dir_name+'/*.csv')):
    df = pd.read_csv(file_name,names=['close'])
    dataset_name = file_name.split("/")[1].split(".csv")[0]
    print(dataset_name)
    stocks[dataset_name] = df
  return stocks

stocks = load_df_from_csv(dir_name="tickers_data")



خچرخش
شپنا


In [27]:
for key, value in stocks.items():
  df = stocks[key]
  close_data = df.close.values.tolist()
  print(close_data)

['close', '3860.0', '3863.0', '4056.0', '4157.0', '4000.0', '4200.0', '4410.0', '4630.0', '4861.0', '5104.0', '5359.0', '5626.0', '5907.0', '6202.0', '6512.0', '6674.0', '6840.0', '7011.0', '7186.0', '7365.0', '7549.0', '7624.0', '7700.0', '7777.0', '7854.0', '7932.0', '8011.0', '8091.0', '8171.0', '8252.0', '8334.0', '8417.0', '8501.0', '8586.0', '8671.0', '8757.0', '8844.0', '8932.0', '17425.0', '17860.0', '18306.0', '18763.0', '18763.0', '18740.0', '18738.0', '18738.0', '17802.0', '16912.0', '16067.0', '16870.0', '16915.0', '17760.0', '17760.0', '17760.0', '17760.0', '17758.0', '17000.0', '16985.0', '17000.0', '17000.0', '16999.0', '17000.0', '17000.0', '17000.0', '17028.0', '17029.0', '17029.0', '17029.0', '17100.0', '17200.0', '17202.0', '17204.0', '17400.0', '17400.0', '17450.0', '17450.0', '17447.0', '16625.0', '15794.0', '12297.0', '12911.0', '13556.0', '14233.0', '14306.0', '14500.0', '15009.0', '15759.0', '15763.0', '15765.0', '15765.0', '15700.0', '15699.0', '15200.0', '1520

In [ ]:
import warnings
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA
import pytse_client as tse


TRAINING_PERCENTAGE = 0.7
TESTING_PERCENTAGE = 1 - TRAINING_PERCENTAGE
NUMBER_OF_PREVIOUS_DATA_POINTS = 3
LENGTH_DATA_SET = 0
TRAINING_SET_LENGTH = 0
TESTING_SET_LENGTH = 0


def training_testing_buckets(raw_data, training_percentage, testing_percentage):
    global TRAINING_SET_LENGTH, TESTING_SET_LENGTH
    TRAINING_SET_LENGTH = int(LENGTH_DATA_SET * training_percentage)
    TESTING_SET_LENGTH = LENGTH_DATA_SET - TRAINING_SET_LENGTH
    training_set, testing_set = raw_data[0:TRAINING_SET_LENGTH], raw_data[TRAINING_SET_LENGTH:LENGTH_DATA_SET]
    return training_set, testing_set


def evaluate_performance_arima(testing_actual, testing_predict):
    counter = 0
    for i in range(len(testing_actual)-1):
        predict=testing_predict[i+1]- testing_predict[i]
        actual = testing_actual[i+1]-testing_actual[i]
        if (actual > 0 and predict > 0) or (actual < 0 and predict < 0):
            counter+=1
    counter = round((counter / (len(testing_actual)-1))*100,2)
    return counter
    # return mean_squared_error(testing_actual, testing_predict)


def plot_arima(currency, testing_actual, testing_predict, file_name):
    actual = pyplot.plot(testing_actual, label="Actual data points", color="blue")
    testing = pyplot.plot(testing_predict, label="Testing prediction", color="green")
    pyplot.ylabel('currency values for 1 USD')
    pyplot.xlabel('number of days')
    pyplot.title( currency + ' : actual vs predicted ')
    pyplot.legend()
    pyplot.show()
    pyplot.savefig(file_name)
    pyplot.clf()


def load_data_set(currency):
    # tse.download(symbols=stock_name)
    ticker = tse.Ticker(stock_name)
    data = ticker.history
    raw_data = data.close.values.tolist()
    global LENGTH_DATA_SET
    LENGTH_DATA_SET = len(raw_data)
    return raw_data


def build_model_predict_arima(training_set, testing_set):
    testing_predict = list()
    training_predict = list(training_set)
    for testing_set_index in range(TESTING_SET_LENGTH):
        arima = ARIMA(training_predict, order=(5, 1, 0))
        arima_model = arima.fit(disp=0)
        forecasting = arima_model.forecast()[0].tolist()[0]
        testing_predict.append(forecasting)
        training_predict.append(testing_set[testing_set_index])
        # print("Predicted = ", testing_predict[-1], "Expected = ", testing_set[testing_set_index])
    print('predicting...')
    print("---------------------------")
    forcast = arima_model.forecast()[0]
    print('The prediction for the next day:', forcast)
    if forcast- testing_set[-1] > 0 : print("last result = +1")
    else :  print("last result = -1")
    print("---------------------------")
    return testing_predict


def arima_model(currency):
    print('\nARIMA Model')

    print('loading the dataset...')
    raw_data = load_data_set(currency)

    print('splitting training and testing set...')
    training_actual_arima, testing_actual_arima = training_testing_buckets(raw_data, TRAINING_PERCENTAGE,
                                                                           TESTING_PERCENTAGE)

    print('building and training model...')
    testing_predict_arima = build_model_predict_arima(training_actual_arima, testing_actual_arima)

    print('evaluating performance...')
    mse_arima = evaluate_performance_arima(testing_actual_arima, testing_predict_arima)
    print('Testing Accuracy: ', mse_arima,"%")

    with open("mse_arima.txt", 'w') as mse_file:
        mse_file.write(str(mse_arima) + '\n')
    print("------------------------")
    print('plotting the graph...')
    plot_arima(currency, testing_actual_arima, testing_predict_arima, "testing_prediction_arima.pdf")

    print('done...')
    return raw_data, testing_predict_arima


if __name__ == '__main__':
    warnings.filterwarnings("ignore")
    # stock_name = input('Enter stock name:')
    stock_name = 'شپنا'
    arima_model(stock_name)  # setting the entry point
